# Main script to clean PDD data

Modules: N/A <br>
Author: Cornelia Ilin <br>
Email: cilin@ischool.berkeley.edu <br>
Date created: March 28, 2022 <br>

### Step 1: Import packages

In [ ]:
import pandas as pd
import numpy as np
import os

### Step 2: Define working directories

In [ ]:
in_dir = 'C:/Users/cilin/Research/CA_hospitals/Input/interm_data/health/'
in_dir_data_selection = 'C:/Users/cilin/Research/CA_hospitals/Input/raw_data/health/'
out_dir = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/health/'

### Step 3: Define functions

``read data``

In [ ]:
def read_data():
    ''''''
    df = pd.read_csv(
        in_dir + 'PDD.csv'
    )
    
    # add source of data
    df['data_source'] = 'PDD'
    
    return df

---
preprocessing - add, recode, substitute

---

``dates``

In [ ]:
def add_dates(df):
    ''' Add dates for year, month, day of birth for patient
        Add dates for year, month, day of hospital visit for patient
    '''
    # define dates
    dates = ['bthdate', 'admtdate']
    
    for col in dates:
        # transform to string
        df[col] = df[col].astype(str)
        # make sure date is Pandas compatible
        df[col] = pd.to_datetime(df[col], errors = 'coerce')

    # define bth variable to be added (year, month, day of birth)
    newvars = [['bthyear', 'bthmonth', 'bthday'],
              ['admtyear', 'admtmonth', 'admtday']]
    
    for i in range(len(dates)):
        # add bth year
        df[newvars[i][0]] = pd.DatetimeIndex(df[dates[i]]).year
        # add bth month
        df[newvars[i][1]] = pd.DatetimeIndex(df[dates[i]]).month
        # add bth date
        df[newvars[i][2]] = pd.DatetimeIndex(df[dates[i]]).day
    
    return df

``zip``

In [ ]:
def recode_zip(df):
    ''' Recode zip of patient
    '''
    
    # define zipcode variables
    zips = ['patzip', 'hplzip']
    
    for val in zips:
        # recode zip as string
        df[val] = df[val].astype(str)
        
        # recode XXXXX, YYYYY, ZZZZZ
        df[val] = np.where(df[val].eq('XXXXX'), 'nan',
                          np.where(df[val].eq('YYYYY'), 'outside of US',
                                  np.where(df[val].eq('ZZZZZ'), 'homeless', df[val])))
        
        # set zip to 'nan' depending on zip length
        df['len_zip'] = df[val].str.len()
        df[val] = np.where(df['len_zip'].isin((1, 2, 4, 6)), 'nan', df[val])

            
        # remove .0 or 0000.0 from zip code if it has any
        df[val] = np.where(df['len_zip'].isin((7,11)), df[val].str[:5], df[val])
        
        # drop len_zip
        df.drop(columns=['len_zip'], inplace=True)
        
    return df

``county``

In [ ]:
def recode_county(df):
    ''' Recode county of patient
    '''

    cols = ['patcnty', 'hplcnty']
    for col in cols:
        # transform to string
        df[col] = df[col].astype(str)

        # read county code and associated names from the data_selection.xlsx file
        cnty_values = pd.read_excel(
            in_dir_data_selection + 'data_selection.xlsx',
            'County_names', skiprows = 2, header = 0
        ).iloc[:,1:3] # select only the first 2 columns

        cnty_values = cnty_values.astype(str)
        cnty_values['county_code'] = cnty_values.county_code + '.0'

        # add county names to df
        temp_df = df[[col]].merge(
            cnty_values,
            left_on=col,
            right_on='county_code',
            how='left'
        )

        # rename county_name
        temp_df.rename(
            columns={'county_name': col+'_name'},
            inplace=True
        )

        # replace values in col+'_name' depending on val in col or col+'_name'
        temp_df[col+'_name'] = np.where(temp_df[col].eq('0.0'), 'unknown/outside CA/homeless',
                                  np.where(temp_df[col+'_name'].isna(), 'nan', temp_df[col+'_name']))
        
        # add col+'_name' to original df
        df[col+'_name'] = temp_df[col+'_name']
        
    return df

``diagnosis codes``

In [ ]:
def recode_diagnosis_codes(df):
    '''
    '''
    columns = [
        'diag00', 'diag01', 'diag02', 'diag03', 'diag04',
        'proc00', 'proc01', 'proc02', 'proc03', 'proc04'
    ]

    for col in columns:
        df[col] = df[col].astype(str)
        
    return df

``zip geometry``

In [ ]:
def add_zip_geometry(df):
    ''' Add zip code geometries
    '''

    ## read/preprocess geometry ##
    ##############################
    os.chdir("C:/Users/cilin/Research/CA_hospitals/Script/ssn_selection/cleaning/")
    %run "4. geom_cleaning.ipynb"
    
    # drop geometry column
    gdf_zcta.drop(
        columns='ZCTA10_geometry',
        inplace=True
    )

    ## read/preprocess crosswalk ZIP to ZCTA ##
    ###########################################
    # read crosswalk
    cw= pd.read_csv(
        'C:/Users/cilin/Research/CA_hospitals/Input/raw_data/census_geo/ZiptoZcta_Crosswalk_2021.csv'
    )

    # keep if state is CA
    cw = cw[cw.STATE.eq('CA')]

    # transform to string
    cw['ZIP_CODE'] = cw.ZIP_CODE.astype(str)


    ## add geography to ZIP ##
    ##########################
    # define zip columns
    columns = ['patzip', 'hplzip']  # add pat hospital zipcode
    for idx, col in enumerate(columns):
        print(col)

        ## preprocess df ##
        ###################
        # transform zipI to string    
        df[col] = df[col].astype(str)
        df[col] = df[col].str.split('.').str[0] # remove .0

        # grab P, hP initials
        if idx==0:
            initial='P'
        if idx==1:
            initial='hP'

        ## read unique ZIP in df 
        temp_df = pd.DataFrame(
            df[col].unique(),
            columns=[col]
        )


        # attach ZCTA10 from gdf_zcta file #
        ####################################
        temp_df = temp_df.merge(
            gdf_zcta[['ZCTA10']], 
            left_on=col,
            right_on='ZCTA10',
            how='left'
        )

        # attach ZCTA from crosswalk file #
        ###################################
        temp_df = temp_df.merge(
            cw[['ZIP_CODE', 'ZCTA']], 
            left_on=col,
            right_on='ZIP_CODE',
            how='left'
        )


        # substitute with ZCTA if ZCTA10 is missing
        temp_df['ZCTA10'] = np.where(temp_df.ZCTA10.isna(), temp_df.ZCTA, temp_df.ZCTA10)

        # drop duplicates 
        temp_df.drop_duplicates(
            [col],
            inplace=True
        )


        # add in geometry #
        ###################
        temp_df = temp_df.merge(
            gdf_zcta, 
            on='ZCTA10',
            how='left'
        )


        # drop cols that are not of interest
        temp_df.drop(
            columns=['ZIP_CODE', 'ZCTA'],
            inplace=True
        )


        # merge to original df
        temp_df = df[[col]].merge(
            temp_df,
            on=col,
            how='left'
        )


        # rename columns 
        new_cols = list(temp_df.columns[1:])
        for new_col in new_cols:
            if len(new_col.split('_'))==1:
                temp_name = new_col.split('_')[0]+initial
            else:
                temp_name = new_col.split('_')[0]+initial+'_'+new_col.split('_')[1]
            temp_df.rename(
                columns={new_col:temp_name},
                inplace=True
            )

        # drop col
        temp_df.drop(columns=[col], inplace=True)

        # add temp_df cols to original df
        for temp_col in temp_df.columns:
            df[temp_col] = temp_df[temp_col]

        
    # if ZCTA geometry of ZCTA10P is missing subsistiute with that of hospital
    colsP = ['ZCTA10P', 'ZCTA10P_centroid']
    colshP = ['ZCTA10hP', 'ZCTA10hP_centroid']
    
    for idx3, colP in enumerate(colsP):
        df[colP] = np.where(df[colP].isna(), df[colshP[idx3]], df[colP])
        
    return df      

``keys``

In [ ]:
def add_keys(df):
    ''''''
    # making sure all vars are strings and strip .0
    for col in ['admtyear', 'admtmonth', 'ZCTA10P', 'patcnty']:
        df[col] = df[col].astype(str).str.split('.').str[0]
    
    # create admtyear_ZCTA10P
    df['admtyear_ZCTA10P'] = df.admtyear + '_' + df.ZCTA10P
    
    # create admtyear_patcnty
    df['admtyear_patcnty'] = df.admtyear + '_' + df.patcnty
    
    return df

---
preprocessing - drop

---

``bthdate ge(1991) only``

In [ ]:
def bthdate_ge1991_only(df):
    '''
    '''
    # keep only if bthyear >=1991
    mini_df = df[df.bthyear.ge(1991.)]

    # reset index
    mini_df.reset_index(
        drop=True,
        inplace=True
    )
    
    return mini_df

``cols of interest``

In [ ]:
def keep_cols_of_interest(mini_df):
    '''
    '''
    cols = [
        'pat_id', 'rln',
        'patzip', 'ZCTA10P',
        'patcnty', 'patcnty_name',
        'hplzip', 'ZCTA10hP',
        'hplcnty', 'hplcnty_name',
        'bthdate', 'bthyear', 'bthmonth', 'bthday',
        'admtdate', 'admtyear', 'admtmonth', 'admtday',
        'admtyear_ZCTA10P', 'admtyear_patcnty', 
        'charge', 
        'diag00', 'diag01', 'diag02', 'diag03', 'diag04',
        'proc00', 'proc01', 'proc02', 'proc03', 'proc04',
        'data_source'
    ]

    return mini_df[cols]

### Step 4: Read data

In [ ]:
df = read_data()
print('Shape of data:', df.shape)
df.head(2)

### Step 5: Data preprocessing - add, recode

``dates``

In [ ]:
# add dates
df = add_dates(df)

``zip``

In [ ]:
df = recode_zip(df)

``county``

In [ ]:
df = recode_county(df)

``diagnosis codes``

In [ ]:
df = recode_diagnosis_codes(df)

``zip geometry``

In [ ]:
df = add_zip_geometry(df)

``add keys for merging or FE``

In [ ]:
df = add_keys(df)

In [ ]:
print(
    'Shape of data after Step 5: "Data preprocessing - add, recode":',
    df.shape
)

---
### Step 6: Data preprocessing - drop

``birthyear ge(1991) only``

In [ ]:
mini_df = bthdate_ge1991_only(df)

In [ ]:
print(
    'Shape of data after Step 6: "Data preprocessing - drop":',
    mini_df.shape
)

### Step 7: Export data

In [ ]:
keep_cols_of_interest(mini_df).to_csv(out_dir + 'PDD_final.csv')

<span style="color:pink">[IMPORTANT: still need to deal with payment category and admission source variables, etc]</span>